In [1]:
import ipywidgets as widgets
from sqlalchemy import create_engine
import pandas as pd
import json



In [3]:

# Create SQLite database engine
engine = create_engine("sqlite:///../toots.db")

# Execute SQL query to fetch the toots data
query = """SELECT *
FROM toots
JOIN algo_tags
  ON toots.id = algo_tags.toot_id
"""
df = pd.read_sql(query, engine)

# Display the dataframe
print(df)


    id                          author   
0    1            hagestolz@troet.cafe  \
1    2    ttrpgblogs@chirp.enworld.org   
2    3            gilesgoat@toot.wales   
3    4     AndrewBrandt@sportsbots.xyz   
4    5                   Shuukyoku1995   
5    6   BarstoolBigCat@sportsbots.xyz   
6    7          kohane@cupoftea.social   
7    8      NashvilleSC@sportsbots.xyz   
8    9       greyglass@hcommons.social   
9   10    mariellequinton@ottawa.place   
10  11       Purmelchen@embassy.social   
11  12        MinervasRefuge@tech.lgbt   
12  13   ColoradoRapids@sportsbots.xyz   
13  14         alexeft@mastodon.online   
14  15                  elsie@girlc.at   
15  16                  ai@misskey.art   
16  17  iembot_mqt@masto.globaleas.org   
17  18             Sharr0w@mastodon.ie   
18  19          ZacBelado@hachyderm.io   
19  20    trimet@transit.alerts.social   

                                              content   
0   <p><a href="https://youtube.com/watch?v=hZip_J...  \
1  

In [6]:
# Assuming the dataframe is already defined as 'df'
# Expand the JSON in the 'tags' column
df['tags'] = df['tags_json'].apply(json.loads)
df_expanded = pd.json_normalize(df['tags'])

# Concatenate the expanded columns with the original dataframe
df = pd.concat([df, df_expanded], axis=1)

# Display the updated dataframe
print(df)


    id                          author   
0    1            hagestolz@troet.cafe  \
1    2    ttrpgblogs@chirp.enworld.org   
2    3            gilesgoat@toot.wales   
3    4     AndrewBrandt@sportsbots.xyz   
4    5                   Shuukyoku1995   
5    6   BarstoolBigCat@sportsbots.xyz   
6    7          kohane@cupoftea.social   
7    8      NashvilleSC@sportsbots.xyz   
8    9       greyglass@hcommons.social   
9   10    mariellequinton@ottawa.place   
10  11       Purmelchen@embassy.social   
11  12        MinervasRefuge@tech.lgbt   
12  13   ColoradoRapids@sportsbots.xyz   
13  14         alexeft@mastodon.online   
14  15                  elsie@girlc.at   
15  16                  ai@misskey.art   
16  17  iembot_mqt@masto.globaleas.org   
17  18             Sharr0w@mastodon.ie   
18  19          ZacBelado@hachyderm.io   
19  20    trimet@transit.alerts.social   

                                              content   
0   <p><a href="https://youtube.com/watch?v=hZip_J...  \
1  

In [7]:
df

,id,author,content,created_at,url,raw_blob,tagger_version,toot_id,tags_json,tags,...,is_academic,is_news,is_cringe,is_based,is_israel_palestine,is_joke,is_art,is_trolling,is_opinion,is_informative
0,1,hagestolz@troet.cafe,"<p><a href=""https://youtube.com/watch?v=hZip_J...",2024-01-13 21:17:40.000000,https://troet.cafe/@hagestolz/111750719781349092,"{""id"": 111750719799254448, ""created_at"": ""2024...",0.1,111750719799254448,"{""topic"":""Roland Schaeffer posted"",""is_polemic...","{'topic': 'Roland Schaeffer posted', 'is_polem...",...,False,False,False,False,False,False,False,False,False,True
1,2,ttrpgblogs@chirp.enworld.org,"<p>New <a href=""https://chirp.enworld.org/tags...",2024-01-13 21:17:39.000000,https://chirp.enworld.org/@ttrpgblogs/11175071...,"{""id"": 111750719830178478, ""created_at"": ""2024...",0.1,111750719830178478,"{""topic"":""TTRPG Blogs posted: New #dnd / #rpg ...",{'topic': 'TTRPG Blogs posted: New #dnd / #rpg...,...,False,False,False,False,False,False,False,False,False,True
2,3,gilesgoat@toot.wales,"<p>Maybe you know it already, maybe you don't,...",2024-01-13 21:17:40.000000,https://toot.wales/@gilesgoat/111750719780281077,"{""id"": 111750719840150091, ""created_at"": ""2024...",0.1,111750719840150091,"{""topic"":""Giles Goat"",""is_polemical"":false,""is...","{'topic': 'Giles Goat', 'is_polemical': False,...",...,False,False,False,False,False,False,False,False,False,True
3,4,AndrewBrandt@sportsbots.xyz,"<p>Tomorrow at this time..<a href=""https://twi...",2024-01-13 21:16:02.000000,https://twitter.com/AndrewBrandt/status/174627...,"{""id"": 111750719862228010, ""created_at"": ""2024...",0.1,111750719862228010,"{""topic"":""Andrew Brandt 🤖 posted"",""is_polemica...","{'topic': 'Andrew Brandt 🤖 posted', 'is_polemi...",...,False,False,False,False,False,False,False,False,False,True
4,5,Shuukyoku1995,<p>EP.17『Nebula Knights:宇宙忍者来襲 Ⅶ』 | YUYA <a hr...,2024-01-13 21:17:42.769000,https://mastodon.social/@Shuukyoku1995/1117507...,"{""id"": 111750719914962914, ""created_at"": ""2024...",0.1,111750719914962914,"{""topic"":""YUYA/小熊 posted: EP.17『Nebula Knight...",{'topic': 'YUYA/小熊 posted: EP.17『Nebula Knigh...,...,False,False,False,False,False,False,True,False,False,True
5,6,BarstoolBigCat@sportsbots.xyz,<p>True. Weather is usually very very predicta...,2024-01-13 21:15:46.000000,https://twitter.com/BarstoolBigCat/status/1746...,"{""id"": 111750719938207438, ""created_at"": ""2024...",0.1,111750719938207438,"{""topic"":""weather"",""is_polemical"":false,""is_po...","{'topic': 'weather', 'is_polemical': False, 'i...",...,False,False,False,True,False,False,False,False,False,True
6,7,kohane@cupoftea.social,"<p>""Winter's heart"" <br><a href=""https://cupo...",2024-01-13 21:17:39.000000,https://cupoftea.social/@kohane/11175071970855...,"{""id"": 111750719954493585, ""created_at"": ""2024...",0.1,111750719954493585,"{""topic"":""Kohane-san posted: \""Winter's heart\...","{'topic': 'Kohane-san posted: ""Winter's heart""...",...,False,False,False,False,False,False,True,False,False,False
7,8,NashvilleSC@sportsbots.xyz,"<p>Little chilly this morning, eh <span class=...",2024-01-13 21:16:00.000000,https://twitter.com/NashvilleSC/status/1746279...,"{""id"": 111750719955603034, ""created_at"": ""2024...",0.1,111750719955603034,"{""topic"":""Nashville SC"",""is_polemical"":false,""...","{'topic': 'Nashville SC', 'is_polemical': Fals...",...,False,False,False,False,False,False,False,False,False,True
8,9,greyglass@hcommons.social,<p>Excerpts from a 19th cent. English-Chinese ...,2024-01-13 21:17:43.000000,https://hcommons.social/@greyglass/11175071994...,"{""id"": 111750719971115120, ""created_at"": ""2024...",0.1,111750719971115120,"{""topic"":""qua bird"",""is_polemical"":false,""is_p...","{'topic': 'qua bird', 'is_polemical': False, '...",...,True,False,False,False,False,False,False,False,False,True
9,10,mariellequinton@ottawa.place,<p>Snowshoeing success!</p>,2024-01-13 21:17:42.000000,https://ottawa.place/@mariellequinton

ToggleButton(value=False, description='is_cringe = True', icon='check', tooltip='is_cringe = True')